**Purpose**: In Category Propensity project we need to output only 10 items if users does not purchase **all 10 items** in the past **30 days**, otherwise the model will add to 5 more recommended items. This script simulates how to do.

In [1]:
import pandas as pd
import numpy as np

In [5]:

final = pd.DataFrame({'cmd_id':['002meRU','002meRU','002meRU','002meRU','1TBophvv','1TBophvv','1TBophvv','1TBophvv','lrNPhrY','lrNPhrY','lrNPhrY','lrNPhrY','002vvqK','002vvqK','002vvqK','002vvqK'],
                     'category':['Lamer','Channel','Gucci','LBV','Sartoria Vestrucci','AndAgain','LBV','Peggy Jennings','Gucci','LBV','Sartoria Vestrucci','AndAgain','LBV','Sartoria Vestrucci','AndAgain','Gucci'],
                     'propensity_score':[14.1,12.5,10,8, 27.4,24.4,18.6,15.8,13,11,9,5,15,12,10,6],
                     'propensity_rank':[1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4],
                     'next_best_rank':[np.nan,1,2,3,1,2,np.nan,3,np.nan,np.nan,1,2,1,2,3,4]})
final

,cmd_id,category,propensity_score,propensity_rank,next_best_rank
0,002meRU,Lamer,14.1,1,NaN
1,002meRU,Channel,12.5,2,1.0
2,002meRU,Gucci,10.0,3,2.0
3,002meRU,LBV,8.0,4,3.0
4,1TBophvv,Sartoria Vestrucci,27.4,1,1.0
5,1TBophvv,AndAgain,24.4,2,2.0
6,1TBophvv,LBV,18.6,3,NaN
7,1TBophvv,Peggy Jennings,15.8,4,3.0
8,lrNPhrY,Gucci,13.0,1,NaN
9,lrNPhrY,LBV,11.0,2,NaN


In [6]:
extend_df= final.next_best_rank.isnull().groupby([final['cmd_id']]).sum().astype(int).reset_index(name='null_count')
extend_df

,cmd_id,null_count
0,002meRU,1
1,002vvqK,0
2,1TBophvv,1
3,lrNPhrY,2


In [7]:
keep_list= list(extend_df[extend_df.null_count < 2]['cmd_id'])
keep_list

['002meRU', '002vvqK', '1TBophvv']

### Sample rule ( we will apply n =10 for real data)   
retrieve n= 2 rows per customer if items purchased less than 3 **or** 4 rows if item purchased >= 3 

In [8]:
n = 2     # number of recommended items required by ALS
extra = 2 # number of recommended if the first n items (n rows) purchased

In [9]:
final[(final['cmd_id'].isin(keep_list) & (final['propensity_rank'] <= n)) | ( np.logical_not(final['cmd_id'].isin(keep_list)) & (final['propensity_rank']<= n+ extra)) ]


,cmd_id,category,propensity_score,propensity_rank,next_best_rank
0,002meRU,Lamer,14.1,1,NaN
1,002meRU,Channel,12.5,2,1.0
4,1TBophvv,Sartoria Vestrucci,27.4,1,1.0
5,1TBophvv,AndAgain,24.4,2,2.0
8,lrNPhrY,Gucci,13.0,1,NaN
9,lrNPhrY,LBV,11.0,2,NaN
10,lrNPhrY,Sartoria Vestrucci,9.0,3,1.0
11,lrNPhrY,AndAgain,5.0,4,2.0
12,002vvqK,LBV,15.0,1,1.0
13,002vvqK,Sartoria Vestrucci,12.0,2,2.0
